# Building conversational AI with the Rasa stack
![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTaX3LNhGcAe1HnPZSuWS0oH6af0LJHXcH7If1sQgLCFAT1chNGFg)


This notebook is a basis for my workshop at PyData 2018 Berlin. If you have any questions or would like to learn more about anything included in this notebook, please let me know or get in touch by juste@rasa.com.

In this workshop we are going to build a chatbot capable of checking in on people's mood and take the necessary actions to cheer them up. 


The tutorial consists of three parts:


*   Part 0: Installation and setup
*   Part 1: Teaching the chatbot to understand user inputs using Rasa NLU model
*   Part 2: Teaching the chatbot to handle multi-turn conversations using dialogue management model.
*   Part 3: Resources and tips

## Part 0: Installation

### Let's start with jupyter configuration

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

### Installation of Rasa
Let's start with the installation of Rasa NLU, Rasa Core and a spacy language model. If you have already installed, you can skip this step. 

In [5]:
import sys
python = sys.executable

# In your environment run:
!{python} -m pip install -U rasa_core rasa_nlu[spacy];

# as well as install a language model:
!{python} -m spacy download en_core_web_md
!{python} -m spacy link en_core_web_md en --force;

    100% |████████████████████████████████| 215kB 4.6MB/s ta 0:00:01
    100% |████████████████████████████████| 153kB 3.1MB/s ta 0:00:011
    100% |████████████████████████████████| 17.3MB 654kB/s eta 0:00:011 day, 23:59:59    21% |██████▉                         | 3.7MB 12.3MB/s eta 0:00:02    65% |█████████████████████           | 11.3MB 19.1MB/s eta 0:00:01    70% |██████████████████████▋         | 12.2MB 16.1MB/s eta 0:00:01
    100% |████████████████████████████████| 327kB 8.0MB/s ta 0:00:011
    100% |████████████████████████████████| 942kB 9.3MB/s eta 0:00:01
    100% |████████████████████████████████| 83.1MB 142kB/s ta 0:00:011  0% |                                | 153kB 8.1MB/s eta 0:00:11    0% |▏                               | 491kB 6.0MB/s eta 0:00:14    4% |█▍                              | 3.6MB 6.0MB/s eta 0:00:14    5% |██                              | 5.0MB 6.2MB/s eta 0:00:13    12% |████                            | 10.4MB 6.6MB/s eta 0:00:12    13% |████▏       

    100% |████████████████████████████████| 5.5MB 3.5MB/s ta 0:00:011   15% |█████                           | 870kB 1.6MB/s eta 0:00:03    18% |█████▉                          | 1.0MB 2.8MB/s eta 0:00:02    46% |███████████████                 | 2.6MB 2.2MB/s eta 0:00:02    62% |███████████████████▉            | 3.4MB 2.6MB/s eta 0:00:01    77% |████████████████████████▉       | 4.2MB 3.1MB/s eta 0:00:01    83% |██████████████████████████▊     | 4.6MB 2.6MB/s eta 0:00:01
    100% |████████████████████████████████| 112kB 10.7MB/s a 0:00:01
    100% |████████████████████████████████| 51kB 9.7MB/s eta 0:00:01
    100% |████████████████████████████████| 26.6MB 593kB/s ta 0:00:011 5% |██                              | 1.6MB 2.8MB/s eta 0:00:09    10% |███▍                            | 2.8MB 2.4MB/s eta 0:00:10    12% |████                            | 3.4MB 2.9MB/s eta 0:00:09    16% |█████▏                          | 4.3MB 8.5MB/s eta 0:00:03    25% |████████▏                       | 6.7M

    100% |████████████████████████████████| 133kB 2.4MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 11.0MB/s ta 0:00:01
    100% |████████████████████████████████| 25.2MB 461kB/s ta 0:00:011 1% |▍                               | 327kB 2.2MB/s eta 0:00:12    5% |█▉                              | 1.4MB 1.5MB/s eta 0:00:17    12% |████                            | 3.1MB 2.5MB/s eta 0:00:09    13% |████▎                           | 3.4MB 2.2MB/s eta 0:00:10    14% |████▋                           | 3.6MB 1.8MB/s eta 0:00:13    14% |████▊                           | 3.7MB 2.3MB/s eta 0:00:10    15% |█████                           | 3.9MB 2.0MB/s eta 0:00:11    17% |█████▊                          | 4.5MB 2.3MB/s eta 0:00:09    27% |█████████                       | 7.1MB 2.5MB/s eta 0:00:08    28% |█████████▏                      | 7.2MB 2.5MB/s eta 0:00:08    30% |█████████▊                      | 7.7MB 2.5MB/s eta 0:00:08    47% |███████████████▎                | 12.0MB

    100% |████████████████████████████████| 81kB 9.9MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 14.2MB/s ta 0:00:01
    100% |████████████████████████████████| 204kB 2.6MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 3.9MB/s ta 0:00:011
    100% |████████████████████████████████| 122kB 2.3MB/s ta 0:00:01
    100% |████████████████████████████████| 3.1MB 3.2MB/s ta 0:00:01    14% |████▊                           | 460kB 1.8MB/s eta 0:00:02    72% |███████████████████████▍        | 2.3MB 1.8MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 10.9MB/s ta 0:00:01
    100% |████████████████████████████████| 5.3MB 2.1MB/s ta 0:00:011   24% |███████▊                        | 1.3MB 3.5MB/s eta 0:00:02    26% |████████▋                       | 1.4MB 2.7MB/s eta 0:00:02    29% |█████████▍                      | 1.5MB 2.6MB/s eta 0:00:02    51% |████████████████▍               | 2.7MB 1.1MB/s eta 0:00:03    82% |██████████████████████████▌     

    100% |████████████████████████████████| 757kB 5.7MB/s ta 0:00:011
    100% |████████████████████████████████| 51kB 8.1MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 2.5MB/s eta 0:00:01
    100% |████████████████████████████████| 174kB 2.4MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 3.8MB/s ta 0:00:011
    100% |████████████████████████████████| 552kB 1.7MB/s ta 0:00:011
    100% |████████████████████████████████| 317kB 2.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for terminaltables ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
  Running setup.py bdist_wheel for colorclass ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/d1/86/9d/16127127306a92d7fd30267890a5634026c045391979c4c317
  Running setup.py bdist_wheel for webexteamssdk ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/98/09/8a/03b3fcfe0a351b960427e278d87191e9a3065cd2a36b84ab3d
  

  Stored in directory: /home/jovyan/.cache/pip/wheels/48/5d/04/22361a593e70d23b1f7746d932802efe1f0e523376a74f321e
Successfully built terminaltables colorclass webexteamssdk flask-jwt-simple future simplejson gast absl-py termcolor tzlocal ConfigArgParse docopt Twisted ujson regex tabulate wrapt
rasa-nlu 0.14.1 has requirement coloredlogs~=9.0, but you'll have coloredlogs 10.0 which is incompatible.
rasa-nlu 0.14.1 has requirement packaging~=17.1, but you'll have packaging 18.0 which is incompatible.
rasa-nlu 0.14.1 has requirement scikit-learn~=0.20.2, but you'll have scikit-learn 0.20.0 which is incompatible.
  Found existing installation: numpy 1.13.3
    Uninstalling numpy-1.13.3:
      Successfully uninstalled numpy-1.13.3
  Found existing installation: jsonschema 3.0.0a3
    Uninstalling jsonschema-3.0.0a3:
      Successfully uninstalled jsonschema-3.0.0a3
  Found existing installation: cloudpickle 0.5.6
    Uninstalling cloudpickle-0.5.6:
      Successfully uninstalled cloudpickl

    100% |████████████████████████████████| 120.9MB 12.8MB/s a 0:00:011-1 day, 23:59:54                     | 2.1MB 532kB/s eta 0:03:44    2% |█                               | 3.4MB 245kB/s eta 0:07:58    4% |█▎                              | 5.0MB 2.8MB/s eta 0:00:42    7% |██▎                             | 8.5MB 2.8MB/s eta 0:00:41    9% |███                             | 11.2MB 4.3MB/s eta 0:00:26    9% |███                             | 11.7MB 1.1MB/s eta 0:01:41    10% |███▍                            | 12.8MB 3.8MB/s eta 0:00:29    11% |███▊                            | 14.0MB 1.1MB/s eta 0:01:35    14% |████▌                           | 17.1MB 2.1MB/s eta 0:00:50    16% |█████▏                          | 19.4MB 2.3MB/s eta 0:00:45    16% |█████▎                          | 19.9MB 5.9MB/s eta 0:00:18    16% |█████▍                          | 20.4MB 2.2MB/s eta 0:00:46    17% |█████▌                          | 20.6MB 2.9MB/s eta 0:00:35    18% |█████▉                          | 22

Let's test the installation - we should have rasa_nlu: 0.12.3 and rasa_core: 0.9.6 installed, and spacy model should be available.

In [7]:
import rasa_nlu
import rasa_core
import spacy

print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))
print("Loading spaCy language model...")
print(spacy.load("en")("Hello world!"))

rasa_nlu: 0.14.1 rasa_core: 0.13.0
Loading spaCy language model...
Hello world!


### Some additional Tools needed
To do some of the visualizations you will also need graphviz. If you don't have graphviz installed, and this doesn't work: don't worry. I'll show you the graph and besides that visualization everything else will work.

Try installing with anyone of these (or adapt to your operating system):

In [7]:
!sudo apt-get -qq install -y graphviz libgraphviz-dev pkg-config;
#!brew install graphviz

[sudo] password for jovyan: 


and another python package and we are ready to go:

In [6]:
!{python} -m pip install pygraphviz;

    100% |████████████████████████████████| 122kB 1.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for pygraphviz ... error
  Complete output from command /opt/conda/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-4_owlt42/pygraphviz/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/pip-wheel-cnq1_2i0 --python-tag cp36:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.6
  creating build/lib.linux-x86_64-3.6/pygraphviz
  copying pygraphviz/agraph.py -> build/lib.linux-x86_64-3.6/pygraphviz
  copying pygraphviz/release.py -> build/lib.linux-x86_64-3.6/pygraphviz
  copying pygraphviz/version.py -> build/lib.linux-x86_64-3.6/pygraphviz
  copying pygraphviz/__init__.py -> build/lib.linux-x86_64-3.6/pygraphviz
  copying pygraphviz/graphviz.py -> build/lib.linux-x86_64-3.6/pygraphviz
  creating build

In [5]:
!pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip
!pip install sklearn_crfsuite

     \ 13.8MB 52.0MB/ss   - 460kB 2.4MB/s     | 3.2MB 7.4MB/s     - 5.0MB 12.6MB/s     | 6.9MB 8.5MB/s     - 9.9MB 13.6MB/s
    100% |████████████████████████████████| 276kB 4.0MB/s ta 0:00:01
    100% |████████████████████████████████| 1.4MB 10.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for pythainlp ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-rkv8q8ur/wheels/79/4e/1e/26f3198c6712ecfbee92928ed1dde923a078da3d222401cc78
  Running setup.py bdist_wheel for marisa-trie ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
  Running setup.py bdist_wheel for nltk ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/4b/c8/24/b2343664bcceb7147efeb21c0b23703a05b23fcfeaceaa2a1e
Successfully built pythainlp marisa-trie nltk


## Part 1: Natural Language Understanding

At first, let's teach our chatbot how to understand user inputs. To do that, we are going to build a Rasa NLU model. Here is some data to get started: 

**Conversation_1:**   
U: Hello  
B: Hello, how are you doing?  
U: I am doing great!  
B: Great. Carry on!  
    
**Conversation_2:**  
U: Hey  
B: Hello, how are you doing?  
U: I am very sad  
B: To cheer you up, I can show you a cute picture of a cat, a dog or a bird. Choose one :)  
U: A kitten  
B: Here is something to cheer you up. Did that help?  
U: Yes  
B: Goodbye  
  
**Conversation_3:**    
U: Heya  
B: Hello, how are you doing?  
U: Not so good and the only thing that could help me feel better is a picture of a puppy  
B: Here is something to cheer you up. Did that help?  
U: No  
B: Goodbye  
U: Bye  

### Creating the training data for language understanding model


Lets create some training data here, grouping user messages by their `intents`. The intent describes what the messages *mean*. Another important part of training data are `entities` - pieces of information which help a chatbot understand what specifically a user is asking about. Entities are labeled using the markdown link syntex: `[entity value](entity_type)` [More information about the data format](https://nlu.rasa.com/dataformat.html#markdown-format).

In [6]:
nlu_md = """
## intent:greet
- hey
- hello there
- สวีดัด
- หวัดดี

## intent:goodbye
- cu
- good by
- cee you later
- ลาก่อย
- บาย
- บ๊าย บาย
- บัย
"""

%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


### Defining the NLU model

Once the training data is ready, we can define our NLU model. We can do that by constructing the processing pipeline which defines how structured data is extracted from unstructured user inputs. 

In [9]:
config = """
language: "en"

pipeline:
- name: "sentiment.LextoTokenizer"
- name: "ner_crf"
- name: "ner_synonyms"
- name: "intent_featurizer_count_vectors"
- name: "intent_classifier_tensorflow_embedding"

""" 

%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


### Training the Rasa NLU Model

We're going to train a model to recognise user inputs, so that when you send a message like "hello" to your bot, it will recognise this as a `"greet"` intent.

In [10]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.training_data.loading:Training data format of nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 11 (2 distinct intents)
	- Found intents: 'goodbye', 'greet'
	- entity examples: 0 (0 distinct entities)
	- found entities: 

INFO:rasa_nlu.model:Starting to train component tokenizer_pylexto
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component ner_crf
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component ner_synonyms
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_featurizer_count_vectors
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_classifier_tensorflow_embedding
INFO:rasa_nlu.classifiers.embedding_intent_classifier:Accuracy is updated every 10 epochs
Epochs: 100%|██████████| 300/300 [00:04<00:00, 70.41it/s, loss=0.082, acc=1.000]


### Using & evaluating the NLU model

Let's see how the model is performing on some of the inputs:

In [12]:
pprint(interpreter.parse("สวัสดี"))

{
  "intent": {
    "name": "greet",
    "confidence": 0.9627447128295898
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "greet",
      "confidence": 0.9627447128295898
    },
    {
      "name": "goodbye",
      "confidence": 0.0
    }
  ],
  "text": "\u0e2a\u0e27\u0e31\u0e2a\u0e14\u0e35"
}


Instead of evaluating it by hand, the model can also be evaluated on a test data set (though for simplicity we are going to use the same for test and train):

In [ ]:
from rasa_nlu.evaluate import run_evaluation

run_evaluation("nlu.md", model_directory)

# Part 2: Handling the dialogue

We have taught our chatbot how to understand user inputs. Now, it's time to teach our chatbot how to make responses by training a dialogue management model using Rasa Core.

### Writing Stories

The training data for dialogue management models is called `stories`. A story is an actual conversation where user inputs are expressed as intents as well as corresponding entities, and chatbot responses are expressed as actions.


Let's take a look into the format of the stories in more detail:

A story starts with `##` and you can give it a name. 
Lines that start with `*` are messages sent by the user. Although you don't write the *actual* message, but rather the intent (and the entities) that represent what the user *means*. 
Lines that start with `-` are *actions* taken by your bot. In this case all of our actions are just messages sent back to the user, like `utter_greet`, but in general an action can do anything, including calling an API and interacting with the outside world. 

In [ ]:
stories_md = """


"""

%store stories_md > stories.md

### Defining a Domain

The domain specifies the universe that the bot operates in. In chatbot's world this universe consists of intents and entities as well as the actions which appear in training stories. The domain can also contain the templates for the answers a chabot should use to respond to the user and slots which will help the chatbot to keep track of the context. Let's look into the domain of our bot:

In [ ]:
domain_yml = """
intents:
- greet
- goodbye
- mood_affirm
- mood_deny
- mood_great
- mood_unhappy
- inform

    
entities:
- group

actions:
- utter_greet
- utter_did_that_help
- utter_happy
- utter_goodbye
- utter_unclear
- utter_ask_picture


templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_unclear:
  - text: "I am not sure what you are aiming for."
  
  utter_happy:
  - text: "Great carry on!"

  utter_goodbye:
  - text: "Bye"
  
  utter_ask_picture:
  - text: "To cheer you up, I can show you a cute picture of a dog, a cat or a bird. Which one do you choose?"
"""

%store domain_yml > domain.yml

### Adding Custom Actions

The responses of the chatbot can be more than just simple text responses - we can call an API to retrieve some data which can later be used to create a response to user input. Let's create a custom action for our bot which, when predicted, will make an API and retrieve a picture of a dog, a cat or a bird, depending on which was specified by the user. The bot will know which type of picture should be received by retrieving the value of the slot `group`.


In [ ]:
from rasa_core.actions import Action
from rasa_core.events import SlotSet
from IPython.display import Image

import requests

class ApiAction(Action):
    def name(self):
        return "action_retrieve_image"

    def run(self, dispatcher, tracker, domain):
        

        group = 
        r = 
        
        response = r.content.decode()
        response = response.replace('["',"")
        response = response.replace('"]',"")
        
        dispatcher.utter_message("")

### Pro Tip: Visualising the Training Data

You can visualise the stories to get a sense of how the conversations go. This is usually a good way to see if there are any stories which don't make sense


In [ ]:
from IPython.display import Image
from rasa_core.agent import Agent

agent = Agent('domain.yml')
agent.visualize("stories.md", "story_graph.html", max_history=2)

### Training your Dialogue Model

Now we are good to train the dialogue management model. We can specify what policies should be used to train it - in this case, the model is a neural network implemented in Keras which learns to predict which action to take next. We can also tweak the parameters of what percentage of training examples should be used for validation and how many epochs should be used for training.

In [ ]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions


agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy()])

# loading our neatly defined training dialogues
training_data = agent.load_data('stories.md')

agent.train(
    training_data,
    validation_split=0.0,
    epochs=200
)

agent.persist('models/dialogue')

### Starting up the bot (with NLU)

Now it's time for the fun part - starting the agent and chatting with it. We are going to start the `Agent` by loading our just trained dialogue model and using the previously trained nlu model as an interpreter for incoming user inputs.

In [ ]:
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)

### Talking to the Bot (with NLU)

Let's have a chat!

In [ ]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])


### Evaluation of the dialogue model
As with the NLU model, instead of just subjectively testing the model, we can also evaluate the model on a dataset. You'll be using the training data set again, but usually you'd use a test data set separate from the training data.

In [ ]:
from rasa_core.evaluate import run_story_evaluation

run_story_evaluation("stories.md", "models/dialogue", 
                     nlu_model_path=None, 
                     max_stories=None, 
                     out_file_plot="story_eval.pdf")

### Interactive learning
Unfortunately, this doesn't work in Jupyter yet. Hence, we going to do this on the command line. To start the interactive training session open your command line and run `train_online.py` script.

### Resources and tips

- Rasa NLU [documentation](https://nlu.rasa.com/)
- Rasa Core [documentation](https://core.rasa.com/)
- Rasa Community on [Gitter](https://gitter.im/RasaHQ/home)
- Rasa [Blog](https://blog.rasa.com/)